In [1]:
import os

In [2]:
os.chdir("..")
current_dir = os.getcwd()
print("Current directory:", current_dir)

Current directory: /Users/sarah_shoilee/codeProjects/stamboekn_KE


In [3]:
data_path = "data"
output_path = os.path.join(data_path, "tables", "html")

### pagexml2html

In [4]:
from src.utils import pagexml_to_html

for filename in os.listdir(os.path.join(data_path, "images")):
    if not filename.endswith('.jpg'):
        continue
        
    IMAGE_NAME = filename
    directory = os.path.join(data_path, "tables", "pagexml")

    pagexml_file = os.path.join(directory, IMAGE_NAME+ ".xml")
    output_file = os.path.join(output_path, IMAGE_NAME + ".html")
    pagexml_to_html(pagexml_file, output_file)

### html2json

In [1]:
from bs4 import BeautifulSoup
import os
import json
from person_info_extraction import extract_info_regex, extract_info_LLM

def html_to_logical_grid(constructed_html):
    soup = BeautifulSoup(constructed_html, 'html.parser')

    rows = soup.find_all('tr')
    persons = []

    # --- Step 1: make a grid keeping track of rowspans ---
    logical_rows = []
    rowspans = {}  # {(col_idx): [remaining_rows, cell_content]}

    for r_idx, tr in enumerate(rows):
        # Start with any carried over rowspans from previous rows
        current_row = []
        to_remove = []

        # Fill carried-down cells first
        for col_idx, (remaining, cell) in rowspans.items():
            current_row.append(cell)
            rowspans[col_idx][0] -= 1
            if rowspans[col_idx][0] <= 0:
                to_remove.append(col_idx)
        for col_idx in to_remove:
            del rowspans[col_idx]

        # Now add new cells from this row
        c_idx = 0
        for td in tr.find_all('td'):
            while any(k == c_idx for k in rowspans):  # skip columns occupied by carried cells
                c_idx += 1
            text = td.get_text(" ", strip=True)
            cell_id = td.get('id')
            r_idx = int(td.get('row', r_idx))
            c_idx = int(td.get('col', c_idx))
            rowspan = int(td.get('rowspan', 1))
            colspan = int(td.get('colspan', 1))

            cell_data = {
                'text': text,
                'id': cell_id,
                'row': r_idx,
                'col': c_idx,
                'rowspan': rowspan,
                'colspan': colspan
            }

            # Place cell in current row
            current_row.append(cell_data)

            # Store for future rows if rowspan > 1
            if rowspan > 1:
                rowspans[c_idx] = [rowspan - 1, cell_data]

            c_idx += colspan

        logical_rows.append(current_row)

    # --- Step 2: Extract person info from each logical row ---
    for i, row in enumerate(logical_rows):
        person = {}
            
        # print(generate_prompt(cells))
        person = json.loads(extract_info_LLM(row))
        if all(v['value']==None for v in person.values()):
            continue

        if person:
            persons.append(person)

    unique_persons = {json.dumps(person, sort_keys=True) for person in persons}
    unique_persons_list = [json.loads(p) for p in unique_persons]
            
    json_obj = {"persons": unique_persons_list}
    print(json.dumps(json_obj, indent=2, ensure_ascii=False))

    with open(f"data/labels/{filename.replace('.jpg.html', '.json')}", "w", encoding='utf-8') as json_file:
        json.dump(json_obj, json_file, ensure_ascii=False, indent=2)#!/usr/bin/env python3

os.chdir("..")
for filename in os.listdir(os.path.join("data", "labels")):
    if not filename.endswith('.html'):
        continue
    with open(os.path.join("data/labels", filename), 'r', encoding='utf-8') as f:
        label_html = f.read()
        html_to_logical_grid(label_html)

{
  "persons": [
    {
      "geboorte_datum": {
        "cell": "t1c15",
        "value": "4 Juny 1796"
      },
      "geboorte_plaats": {
        "cell": "t1c15",
        "value": "Brunswijk"
      },
      "laatste_woonplaats": {
        "cell": "t1c15",
        "value": "Leyden"
      },
      "moeder": {
        "cell": "t1c15",
        "value": "Melunina Caroline Sophiar geb: Drechler"
      },
      "vader": {
        "cell": "t1c15",
        "value": "Christeaan Nicolaas Luding"
      }
    },
    {
      "geboorte_datum": {
        "cell": "t1c9",
        "value": "14 Junij 1785"
      },
      "geboorte_plaats": {
        "cell": "t1c9",
        "value": "Luxemburg"
      },
      "laatste_woonplaats": {
        "cell": "t1c9",
        "value": "Edem"
      },
      "moeder": {
        "cell": "t1c9",
        "value": "Margaretha Elter"
      },
      "vader": {
        "cell": "t1c9",
        "value": "Jan Nicolaas"
      }
    },
    {
      "geboorte_datum": {
        "ce